# 🌌 P(k) Plot Simulation : HI Map

#### Pragun Nepal  
#### June 26, 2025

---

This notebook contains the code for running the P(k) Plot Simulation for Epoch of Reionization. The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body, FoF-Halo-finder, and ReionYuga Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for P(k) Plot Simulation for HI map](#Code_for_N-body_Simulation)
3. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for P(k) Plot Simulation for HI map

```python
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

#Parameters
interp_steps = 20 
zoom_x_fraction = 0.2
num_k_ticks = 5
y_margin_factor = 1.5
line_width = 4
spine_width = 2.5
tick_width = 2.5
tick_length = 8
label_fontsize = 18
tick_labelsize = 16

#Gather and parse files
files = glob.glob('pk.ionz*_*')

def extract_xhi_z(fname):
    m = re.search(r'pk\.ionz([\d\.]+)_([\d\.]+)$', fname)
    if m:
        xhi, z = float(m.group(1)), float(m.group(2))
        return xhi, z
    return None, None

parsed_files = [(f, *extract_xhi_z(f)) for f in files]
parsed_files = [t for t in parsed_files if t[1] is not None and t[2] is not None]
files_sorted = sorted(parsed_files, key=lambda x: x[2], reverse=True)

#Load and scale valid data
ks_list = []
ps_list = []
xhi_list = []
z_list = []

for fname, xhi, z in files_sorted:
    data = np.loadtxt(fname)
    k, p = data[:, 0], data[:, 1]
    p_scaled = p * (xhi ** 2) * (z ** 2)

    valid_mask = (k > 0) & (p_scaled > 0) & np.isfinite(k) & np.isfinite(p_scaled)
    k = k[valid_mask]
    p_scaled = p_scaled[valid_mask]

    if len(k) == 0 or len(p_scaled) == 0:
        print(f"Skipping {fname}: No valid positive data.")
        continue

    ks_list.append(k)
    ps_list.append(p_scaled)
    xhi_list.append(xhi)
    z_list.append(z)

if len(ks_list) == 0:
    raise RuntimeError("No valid data found after filtering.")

#Determining plotting limits
all_x = np.hstack(ks_list)
all_y = np.hstack(ps_list)
x_min, x_max = all_x.min(), all_x.max()
finite_y = all_y[np.isfinite(all_y) & (all_y > 0)]
y_min = np.percentile(finite_y, 2)
y_max = np.percentile(finite_y, 98) * y_margin_factor
x_range = x_max - x_min
x_zoom_max = x_min + zoom_x_fraction * x_range
k_ticks = np.logspace(np.log10(x_min), np.log10(x_zoom_max), num=num_k_ticks)

#Interpolation
interp_ks = []
interp_ps = []
interp_xhi = []
interp_zs = []

for i in range(len(ps_list) - 1):
    k1, k2 = ks_list[i], ks_list[i + 1]
    p1, p2 = ps_list[i], ps_list[i + 1]
    xhi1, xhi2 = xhi_list[i], xhi_list[i + 1]
    z1, z2 = z_list[i], z_list[i + 1]

    for t in range(interp_steps):
        alpha = t / interp_steps
        interp_ks.append(k1)
        interp_ps.append((1 - alpha) * p1 + alpha * p2)
        interp_xhi.append((1 - alpha) * xhi1 + alpha * xhi2)
        interp_zs.append((1 - alpha) * z1 + alpha * z2)

interp_ks.append(ks_list[-1])
interp_ps.append(ps_list[-1])
interp_xhi.append(xhi_list[-1])
interp_zs.append(z_list[-1])

# Plot setup
fig, ax = plt.subplots(figsize=(32, 13), facecolor='black')
ax.set_facecolor('black')
line, = ax.plot([], [], lw=line_width, color='gold')
title = ax.text(0.5, 1.03, '', transform=ax.transAxes, ha='center', fontsize=20, color='purple', fontweight='bold')

ax.grid(True, color='purple', linewidth=0.5)
ax.xaxis.grid(True, which='major', color='purple', linewidth=0.5)
ax.yaxis.grid(True, which='major', color='purple', linewidth=0.5)

for spine in ax.spines.values():
    spine.set_edgecolor('purple')
    spine.set_linewidth(spine_width)

ax.tick_params(colors='purple', width=tick_width, length=tick_length)
ax.tick_params(labelsize=tick_labelsize)
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight('bold')

ax.xaxis.label.set_color('purple')
ax.yaxis.label.set_color('purple')
ax.xaxis.label.set_size(label_fontsize)
ax.yaxis.label.set_size(label_fontsize)
ax.xaxis.label.set_fontweight('bold')
ax.yaxis.label.set_fontweight('bold')

ax.set_xlabel('k')
ax.set_ylabel('P(k)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xticks(k_ticks)
ax.minorticks_on()
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, pos: f"{val:.2e}"))
ax.yaxis.set_ticks_position('both')
ax.yaxis.set_label_position('left')
ax.set_xlim(x_min, x_zoom_max)
ax.set_ylim(y_min, y_max)

# Animation functions
def init():
    line.set_data([], [])
    title.set_text('')
    return line, title

def animate(i):
    x = interp_ks[i]
    y = interp_ps[i]
    xhi = interp_xhi[i]
    z = interp_zs[i]
    line.set_data(x, y)
    title.set_text(f'HI Map P(k): xₕᵢ = {xhi:.3f}, z = {z:.3f}')
    return line, title

anim = FuncAnimation(
    fig, animate, init_func=init,
    frames=len(interp_ps), interval=100, blit=True
)

writer = FFMpegWriter(fps=interp_steps * 5, bitrate=2000)
anim.save('pkionz_final.mp4', writer=writer)

plt.show()


```
---


## 🔭 <a name="conclusion"></a>3. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal